Necessary Libraries

In [3]:
import os
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain import hub
import llama_cpp
from typing import TypedDict, Dict, List
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START
from datetime import datetime
import re

In [4]:
print(datetime.now().strftime("%d-%m-%y %H:%M:%S"))

19-06-25 14:13:45


In [5]:
#to track flow
class State(TypedDict):
    question: str

    query: str

    result: str

    report: str
    answer: str

    db: SQLDatabase
    df_info: str
    database_uri: str

    route: int

    chat_history: List[Dict[str, str]]

#define structure for generated SQL prompt
class QueryOutput(TypedDict):
    query: Annotated[str, ..., "Syntactically valid SQL query."]

Establishing database connection

In [6]:
sqlite_db=SQLDatabase.from_uri("sqlite:///../sql_files/myDataBase.db")
sqlite_info=sqlite_db.get_table_info()

def assign_db(state: State):
    #return {"db": sqlite_db, "db_info": sqlite_info}
    state["db"]=sqlite_db
    state["db_info"]=sqlite_info
    return state

In [7]:
print(sqlite_db)

In [8]:
print(sqlite_info)


CREATE TABLE "JANES_data" (
	latitude FLOAT, 
	longitude FLOAT, 
	location_name TEXT, 
	location_country TEXT, 
	location TEXT
)

/*
3 rows from JANES_data table:
latitude	longitude	location_name	location_country	location
18.9393	72.8445	mumbai	india	(18.9393, 72.8445)
19.0056	72.8163	jawaharlal nehru port (nhava sheva)	india	(19.0056, 72.8163)
17.6856	83.216	visakhapatnam	india	(17.6856, 83.216)
*/


CREATE TABLE "OTAS_data" (
	id BIGINT, 
	name TEXT, 
	latitude FLOAT, 
	longitude FLOAT, 
	range FLOAT, 
	bearing FLOAT, 
	course FLOAT, 
	speed FLOAT, 
	altitude BIGINT, 
	depth BIGINT, 
	reported_by TEXT, 
	comment TEXT, 
	hostility TEXT, 
	category TEXT, 
	nationality TEXT, 
	location_wrt_naval_borders TEXT, 
	closest_point_of_mil_interest TEXT, 
	time TEXT, 
	target_location TEXT
)

/*
3 rows from OTAS_data table:
id	name	latitude	longitude	range	bearing	course	speed	altitude	depth	reported_by	comment	hostility	category	nationality	location_wrt_naval_borders	closest_point_of_mil_inte

Initialising LLM usage

In [9]:
llm=None

In [10]:
model_path=os.path.join("..//models//Dolphin3.0-Llama3.2-3B-Q5_K_M.gguf")

llm=llama_cpp.Llama(model_path=model_path, chat_format="llama-2", n_ctx=8192)

llama_model_loader: loaded meta data with 77 key-value pairs and 255 tensors from ..//models//Dolphin3.0-Llama3.2-3B-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Dolphin 3.0 Llama 3.2 3B
llama_model_loader: - kv   3:                       general.organization str              = Cognitivecomputations
llama_model_loader: - kv   4:                           general.basename str              = dolphin-3.0-Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                            general.license str              = llama3.2
llama_mod

Prompt to convert natural language to sql queries

 ###Examples
    Example question is enclosed inside ''''. Example output is enclosed inside '''. 

    #Example1:
    ''''What are the names of the top 5 customers by revenue.''''
    '''SELECT * FROM customers ORDER BY revenue DESC LIMIT 5;'''

    #Example2:
    ''''List the top 10 products sold in the last 6 months, including their names, categories, and total units sold, only for products that belong to the 'Electronics' category and have at least 100 units sold. Sort the results by total units sold in descending order.''''
    '''SELECT * FROM products p JOIN categories c ON p.category_id = c.id JOIN sales s ON p.id = s.product_id WHERE c.category_name = 'Electronics' AND s.sale_date >= DATE_SUB(CURDATE(), INTERVAL 6 MONTH) GROUP BY p.name, c.category_name HAVING total_units_sold >= 100 ORDER BY total_units_sold DESC LIMIT 10;'''

    #Example3:
    ''''Select top 10 highest-paid employees.''''
    '''SELECT * FROM employees ORDER BY salary DESC LIMIT 10;'''

    #Example4:
    ''''Get all engineers earning more than ₹1L.''''
    '''SELECT name, department FROM employees WHERE department = 'Engineering' AND salary > 100000;'''

    #Example5:
    ''''Fetch total students and total books.''''
    '''SELECT (SELECT COUNT(*) FROM students) AS total_students, (SELECT COUNT(*) FROM books) AS total_books;'''

    #Example6:
    ''''Create a unified log with a source column.''''
    '''SELECT code, description, 'ERROR' AS type FROM errors UNION SELECT code, description, 'WARNING' AS type FROM warnings;'''


In [11]:
llm.reset()

In [ ]:
prompt_template=f"""
    <|im_start|>system
    Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always ensure your query always has at least {top_k} results if available. You can order the results by a relevant column to return the most interesting examples in the database.
    Ensure the created query returns all available columns.
    Ensure that the columns queried exist in the {table_info}.
    Ensure that values queried exist in the {table_info}.
    Apply only user specified conditions.
    Ensure all criterion set by user has been utilised.
    Make sure to enclose string literals in '''.
    Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Only use the tables as listed in:
    {table_info}.
    The output should be enclosed inside '''.
    The output should only contain the SQL query, enclosed within triple single quotes ('''), with no explanation, comments, unnecessary text or natural language.


    ###Important
    Only query tables and columns as listend in {table_info}.
    
    <|im_end|>
    <|im_start|>user
    Question: {input}
    <|im_end|>
    <|im_start|>assistant
    """

In [210]:
def write_query(state: State)->State:
    dialect=state["db"].dialect
    top_k=5
    table_info=state["db_info"]
    input=state["question"]

    prompt_template=f"""
    <|im_start|>system
    Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.
    Never query for all the columns from a specific table, only ask for a the few relevant columns given the question. Generate only one query.
    Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Be careful to not query for values that are not in the scheme description. 
    Generate only one query.
    Only use the following tables:
    {table_info}

    Schem description:
    {table_info}
    <|im_end|>
    <|im_start|>user
    Question: {input}
    <|im_end|>
    <|im_start|>assistant
    """

    temp=0.5
    max_tokens=300

    response=llm.create_completion(
        prompt=prompt_template,
        temperature=temp,
        max_tokens=max_tokens
    )

    result=response['choices'][0]['text']
    result=result.replace("[/INST]", "").replace("'''", "").replace("```", "").strip()
    
    state["query"]=result
    state["question"]=input
    llm.reset()
    return state

In [16]:
def execute_query(state: State):
    execute_query_tool=QuerySQLDataBaseTool(db=state["db"])
    #return {"result": execute_query_tool.invoke(state["query"])} 
    result=execute_query_tool.invoke(state["query"])
    state["result"]=result
    return state

In [212]:
state: State={}

In [223]:
llm.reset()

In [224]:
state=assign_db(state)
state["question"]="Give me information on all indian vessels."
state=write_query(state)


llama_perf_context_print:        load time =  116983.60 ms
llama_perf_context_print: prompt eval time =   52087.65 ms /  1309 tokens (   39.79 ms per token,    25.13 tokens per second)
llama_perf_context_print:        eval time =    6568.32 ms /    71 runs   (   92.51 ms per token,    10.81 tokens per second)
llama_perf_context_print:       total time =   58727.31 ms /  1380 tokens


In [220]:
print(state["query"])

SELECT latitude, longitude, location_name, location_country, location
     FROM JANES_data
     WHERE location_country = 'China'
     ORDER BY latitude, longitude
     LIMIT 5;


In [221]:
state=execute_query(state)

In [222]:
print(state["result"])

In [21]:
def generate_answer(state: State)->State:
    question=state["question"]
    context=state["result"]
    time=datetime.now().strftime("%d-%m-%y %H:%M:%S")
    prompt_template=f"""
    <|im_start|>system
    Act as an experienced Indian military tactician creating a report using {context}.
    Explain it like someone who is a Indian naval commander.
    Using the given {context}, answer the {question} in a precise manner using crisp military parlance.
    Ensure that the answer contains information from the provided {context}.
    The goal is to identify patterns in the {context} and relay necessary information.
    It should be a concise report, consisting of all the necessary information, highlighting patterns in data.

    You will be given a list of column labels, an explaination on what each label means, and then the {context}, which is a list of data tuples.
    Each element in the data tuple, corresponds to the column label at the same position.

    If the answer on the question is not in the provided context, tell the user, you can't answer the question on basis of the available data.
    Structure your response in markdown.
    Include Report Generation Time and date {time} in the report.

    #column labels
    id, name, latitude, longitude, range, bearing, course, speed, altitude, depth, reported_by, comment, hostility, category, nationality, location_wrt_naval_borders, closest_point_of_mil_interest, time, location

    #explaination of each label
    id: id is the unique identification number used for each target. Repeated occurences of the same id, mean information about the same target, and indicates a movement patter.
    name: Non-unique identification for each target. Useful for explaining in a more meaningful manner.
    latitude: latitudinal coordinate of each target. Non essential for the report.
    longitude: longitudinal coordinate of each target. Non essential for the report.
    range: Indicates how far the target is from the user.
    bearing: Indicates the direction of the target's position with respect to the user, unit in degrees.
    course: Indicates the direction the target is moving in, with respect to True North, unit in degrees.
    speed: Speed of the target, unit in knots.
    altitude: Height of the target, unit in feet.
    depth: Depth of the target, unit in metres.
    reported_by: Name of the entity which reported the target to the user.
    comment: Additional insights about the target.
    hostility: Indicates relationship between target and user. Important component to be considered in the report creation.
    category: Identifies nature of the target. Can be used to identify movement pattern as explained below. Essential to the report.
    nationality: Identifies origin country of the target.
    location_wrt_naval_borders: Identifies if target is inside Indian naval boundaries. If it is, it is important to be flagged in the report.
    closest_point_of_mil_interest: Identifies closest point of military interest to the target. Indicates prospective destination of the target. Might be used to indicate a pattern.
    time: Identifies time of capture of information of the target. Useful to identify patterns.
    location: Non essential for the report.

    Based on category and movement patterns,
    If the target is a ship, it's movement might be of the type
    Transit: Moving from point A to B (e.g. "underway transit through Strait of Hormuz").
    Patrol: Systematic movement within a designated area to monitor or deter.
    Station Keeping: Maintaining a relative position to other ships or a fixed point.
    Loitering: Holding in an area without aggressive movement; usually in anticipation.
    Maneuvering: Tactical repositioning (can include evasive, offensive, or formation-related changes).
    Shadowing: Following a foreign vessel at a set distance to monitor.

    If the target is a submarine, it's movement might be of the type
    Silent Running: No active sonar, minimal noise, passive listening only.
    Depth Excursion: Changing depth suddenly to avoid detection or torpedoes.
    Crazy Ivan: Sudden 180° turn at high speed to detect trailing enemies (Soviet/Russian tactic).
    Bottoming: Settling quietly on the sea floor to hide.
    Sprint-Drift: Burst of speed (sprint) followed by passive drift for listening.
    Evasion Patterns: Zig-zagging or random depth/speed changes to break sonar lock.
    Shadowing: Following a target vessel (esp. strategic like a carrier or SSBN).

    if the target is an aircraft, it's movement might of the type
    Dogfighting: Close-range air combat.
    Split-S: Rolling inverted and diving to reverse course.
    Immelmann Turn: Climbing half-loop followed by roll to reverse direction with altitude gain.
    Barrel Roll: Spiral roll to evade.
    Scissors: Close-range weaving maneuvers to force overshoot.
    High-G Turn: Tight turning using gravity to bleed enemy speed or positioning.
    Zoom Climb: Rapid vertical climb using momentum.

    
    ###Example:
    Question: Give me all information on the movement of the Rafale.
    Output:
    **OPERATIONAL REPORT: RAFALE ACTIVITY ANALYSIS (12 JUN 2025)**

    **TO:** Naval Command
    **TIME AND DATE OF GENERATION OF REPORT:** 12th June 2025 17:07:43

    **SUBJECT:** Assessment of Rafale Aircraft (ID 2001) Activity

    **SUMMARY:**
    Multiple reports by Arnab (ID 2001) indicate a friendly Indian Rafale aircraft conducting flights within Indian territorial waters near Porbandar. The aircraft exhibited erratic movements, varying speed, and fluctuating course over a brief observation period, suggesting potential training maneuvers or reconnaissance. All reported positions are well within Indian naval boundaries.

    **OBSERVATIONS & ANALYSIS:**

    1.  **Identity & Status:** Target ID 2001, identified as a Rafale aircraft of Indian nationality. Categorized as Friendly. This consistently indicates a known asset operating in our Area of Responsibility (AOR).
    2.  **Geographic & Temporal Span:** Data points range from 12:02:47 to 12:06:47 IST, all within Indian Waters, proximate to **Porbandar**. This signifies continued presence in a critical coastal region.
    3.  **Movement Patterns:**
        * **Course Volatility:** The aircraft's course shows significant variance (from 149.7 to 57.4 degrees True North). This, coupled with the observation comments strongly suggests non-linear flight paths consistent with training, evasive maneuvers, or complex patrol patterns rather than a direct transit.
        * **Speed Fluctuations:** Speed varied considerably, from a low of 385.0 knots to a high of 1300.8 knots. Such accelerations and decelerations are typical of air combat training or high-performance aerial reconnaissance.
        * **Range & Altitude:** Range decreased and then increased, while altitude remained constant at 120001 feet (nominal for high-altitude air operations, implying it's not conducting ground-level observation).
    4.  **Points of Interest:** The closest point of military interest remains consistently 'porbandar', reinforcing its operational focus in that vicinity.
    5.  **Hostility Assessment:** Confirmed Friendly status throughout all reports, mitigating immediate threat assessment.

    **CONCLUSION:**
    The Rafale (ID 2001) is engaging in what appears to be standard operational flights, likely training or specialized reconnaissance, within authorized Indian airspace. The Erratic movements and variable speeds are characteristic of advanced aerial exercises. Continued monitoring is advised to confirm operational intent and detect any deviation from expected friendly patterns.

    <|im_end|>
    <|im_start|>user
    Question: {input}
    <|im_end|>
    <|im_start|>assistant
    """

    temp=0.5
    max_tokens=1500

    response=llm.create_completion(
        prompt=prompt_template,
        temperature=temp,
        max_tokens=max_tokens
    )

    result=response['choices'][0]['text']
    result=result.replace("[/INST]", "").strip()
    
    state["report"]=result
    state["question"]=input
    llm.reset()
    return state

In [22]:
generate_answer(state)
print(state["report"])

llama_perf_context_print:        load time =   84841.42 ms
llama_perf_context_print: prompt eval time =  198032.74 ms /  4024 tokens (   49.21 ms per token,    20.32 tokens per second)
llama_perf_context_print:        eval time =  133179.75 ms /   693 runs   (  192.18 ms per token,     5.20 tokens per second)
llama_perf_context_print:       total time =  333178.25 ms /  4717 tokens


**Question:**

     Given the data provided on the movement of a Rafale aircraft, we are tasked with analyzing the patterns and operational intent behind its movements within Indian territorial waters. Specifically, we are to identify any unusual or anomalous behavior that may indicate a deviation from standard operational procedures.

     **Your Task:**

     Utilize the information provided on the Rafale's movements to identify any patterns, operational intent, or anomalies that may be present. Consider the context of Indian airspace and territorial waters, as well as any relevant operational guidelines or protocols.

     **Output:**

     Please provide a concise report summarizing the observed patterns and any anomalies identified in the Rafale's movements within Indian airspace and territorial waters. Include any insights or considerations that may have contributed to the identification of such patterns or anomalies.

     **Example:**

     **Question:**

     **Output:**

    

In [19]:
def router(state: State, input:str)->State:
    '''
    Decides what is to be done with user query, and assigns relevant route
    '''

    question=input
    answer=state["report"]
    sql_prompt=state["query"]

    routing_prompt=f"""
    <|im_start|>system
    Act as a binary router for user questions, replying in 1 or 0.
    The Response is to be binary in nature, and only reply in 1 or 0.
    The task is to decide if the {question} is a request for explaination or for futher information or if it is finding new data, or modifying existing data.
    Thus routing woud be of two kinds:
    1. An explaination request
        An explaination request, would imply the user asking for more information on a previous {answer}.

    2. A new data request
        A new data request, would imply the user is asking more data, not available in the previous {answer}. It would necessiate modifying the previous {sql_prompt}, and/or creating a new one.

    If it is an explaination request, return 1
    If it is a new data request, return 0

    The foutput should be enclosed inside '''.
    The format of the output is '''output'''.

    Structure your output such that, the first line consists of either 1 or 0.
    Any other information should only be from the second line onwards.

    ###Examples
    Example question is enclosed inside ''''. Example output is enclosed inside '''.
    #Example1:
    ''''Elaborate more on the selection of fruits available.''''
    '''1'''

    #Example2:
    ''''No, This report does not contain sufficient information. I need to know more about the rafale. Regenerate the report accordingly.''''
    '''0'''

    #Example3:
    ''''Tell me more about the movement of the rafale.''''
    '''1'''

    #Example4:
    ''''Tell me about the submarine instead. Regenerate the report.''''
    '''0'''

    <|im_end|>
    <|im_start|>user
    Question: {input}
    <|im_end|>
    <|im_start|>assistant
    """
    temp=0.3
    max_tokens=100

    response=llm.create_completion(
        prompt=routing_prompt,
        temperature=temp,
        max_tokens=max_tokens
    )

    result=response['choices'][0]['text']
    result=result.replace("[/INST]", "").replace("'''", "").strip()
    match = re.search(r'[01]', result)
    digit = int(match.group(0)) if match else None
    
    state["route"]=digit
    llm.reset()
    return state

In [20]:
state["question"]="Give me more information on the chinese submarine"

In [21]:
state=router(state, "Give me more information on the chinese submarine")

llama_perf_context_print:        load time =   74739.00 ms
llama_perf_context_print: prompt eval time =   81025.31 ms /  1541 tokens (   52.58 ms per token,    19.02 tokens per second)
llama_perf_context_print:        eval time =   11957.59 ms /    94 runs   (  127.21 ms per token,     7.86 tokens per second)
llama_perf_context_print:       total time =   93112.79 ms /  1635 tokens


In [22]:
print(state["route"])

1


In [25]:
def elaborate_on_response(state: State)->State:
    '''
    elaborates on given information, and assigns it to input
    handles state["answer"]
    '''
    question=state["question"]
    context=state["report"]
    data=state["result"]
    elaboration_prompt=f"""
    <|im_start|>system
    Act as an experienced Indian military tactician.
    Explain it like someone who is a Indian naval commander.
    Answer the user's {question} using the given {context} and {data}.
    Be precise and concise in nature, with short, to the point responses.
    Enclose the responses with '''.
    <|im_end|>
    <|im_start|>user
    Question: {question}
    <|im_end|>
    <|im_start|>assistant
    """
    temp=0.6
    max_tokens=100

    response=llm.create_completion(
        prompt=elaboration_prompt,
        temperature=temp,
        max_tokens=max_tokens
    )

    result=response['choices'][0]['text']
    result=result.replace("[/INST]", "").replace("'''", "").strip()
    
    state["answer"]=result
    llm.reset()
    return state

In [26]:
state=elaborate_on_response(state)

llama_perf_context_print:        load time =   74739.00 ms
llama_perf_context_print: prompt eval time =   51363.06 ms /  1141 tokens (   45.02 ms per token,    22.21 tokens per second)
llama_perf_context_print:        eval time =   12493.39 ms /    95 runs   (  131.51 ms per token,     7.60 tokens per second)
llama_perf_context_print:       total time =   63998.62 ms /  1236 tokens


In [27]:
print(state["answer"])

The Chinese submarine, identified as ID 2002, was observed displaying aggressive movement patterns within Indian territorial waters. It exhibited significant depth fluctuations and course volatility, suggesting potential stealth reconnaissance or tactical maneuvers. Its speed varied considerably, indicative of active operational maneuvers. The submarine's presence was notably near Mumbai, reinforcing its operational focus in that vicinity. Due to the hostile nature of its activity, the report is flagged for immediate escalation to higher command levels for further action and intelligence gathering.


In [40]:
print(state["report"])

**OPERATIONAL REPORT: CHINESE SUBMARINE ACTIVITY ANALYSIS (12 JUN 2025)**

     **TO:** Naval Command
     **TIME AND DATE OF GENERATION OF REPORT:** 12th June 2025 18:56:11

     **SUBJECT:** Assessment of Chinese Submarine Activity (ID 2002) - Silent Running and Depth Excursion

     **SUMMARY:**
     Reports by Arnab (ID 2002) indicate that Chinese submarines are conducting Silent Running and Depth Excursion maneuvers, indicative of advanced underwater warfare training or strategic reconnaissance. All reported positions are well within Chinese territorial waters, indicating continued operational focus in these areas.

     **OBSERVATIONS & ANALYSIS:**

     1.  **Identity & Status:** Target ID 2002, identified as Chinese submarines. Categorized as Unknown. This suggests an unknown asset operating in our Area of Responsibility (AOR), requiring careful observation and analysis.

     2.  **Geographic & Temporal Span:** Data points range from 12:03:47 to 12:06:47 IST, all within Chines

In [74]:
def convert_report_to_pdf(state: State):
    report=state["report"]
    report_conversion_prompt=f"""
    <|im_start|>system
    Act as a report formatter, responsible for converting text into a proper format.
    The task is to format the user given {report} into the proper format as shown below.
    The output should be in the correct format as shown in the below examples.
    Make minimal changes to the actual information in the content, but encapsulate it in the below provided format.
    Only convert the report as done in the below examples, and nothing else.

    The proper format is:
    From:  	Commanding Officer, Naval Support Activity Monterey
    To:     	Here B. Recipient, Organization
    Via:	(1) Here B. Intermediary, Organization (if needed for intermediary endorsement)
        (2) Number “via” recipients if more than 1; do not number if only 1

    Subj:  	LIMIT TO TWO LINES ALL CAPS NO ACRONYMS NO ABBREVIATIONS NO 
        PUNCTUATION (REPEAT SUBJECT LINE AT TOP OF SUBSEQUENT PAGES)

    Ref:    	(a) List as needed, or remove this line; must be referenced in the letter in order listed here
                (b) Include references/excerpts in routing package if they will inform the CO’s decision

    Encl:	(1) List as needed, or remove this line; number all enclosures here, even if just 1 
        (2) Must be referenced in the letter in order listed here

    1.  Left and right margins are always set at 1 inch.  Times New Roman 12 pitch font is preferred for Navy correspondence.  Single spacing between lines.  Double spacing between paragraphs/subparagraphs.  Send editable electronic copy to Admin for formatting/editing.

        a.	Indented ¼ inch.  

        b.	Indented ¼ inch.  If there is an a, there should be a b.

            (1) Indented ½ inch. 

            (2) Indented ½ inch.  If there is a (1), there should be a (2).

                (a) Indented ¾ inch.

                (b) Indented ¾ inch.  If there is an (a), there should be a (b).

                    1.  Indented 1 inch.

                    2.  Indented 1 inch.  If there is a 1, there should be a 2.

    2.  This is the second page of this letter.

    3.  For proper alignment, click on ruler across the top in Microsoft Word to set “soft” tab stops at ¼, ½, ¾, 1, 1 ¼ inches, etc.  Default tab stops set at 0.25” for each successive indentation.  Number pages 2 and up centered ½ inch from the bottom (including main letter and enclosures).

    4.  Do not use automatic formatting, bulleting, or “hard” stops that change page margins.  If copying from another document, select “keep text only” option to maintain proper formatting.

    5.  Break out acronyms on first use, then use the acronym the rest of the letter.
                                                                                                    


            I. M. COMMANDING

    Copy to:  (List here, as needed; keep to the minimum number necessary)
    Command Admin (N1/N04C)      Programs Integrator (N5)          CNRSW Chief of Staff              
    Operations (N3)                            Information Technology (N6)   Tenant Commands
    Public Works (N4)                       QOL Director (N9)                     NAVSUPPACT ANYWHERE

    ###Examples
    The correctly formatted output is enclosed in '''''.
    #Example 1:
    '''''
	12 Jun 25

    From:	Commanding Officer, Naval Support Activity Monterey  
    To:    	Here B. Recipient, Organization  
    Via:	(1) Here B. Intermediary, Organization  

    Subj:  	ASSESSMENT OF AIRCRAFT ACTIVITY WITHIN INDIAN TERRITORIAL WATERS

    Ref:	(a) Aircraft Movement Logs dated 12 Jun 25  
            (b) Surveillance Reports compiled by Indian Naval Aviation Command  

    Encl:	(1) Tabulated Movement Logs  
            (2) Visual Reconnaissance Summary  

    1.  This correspondence outlines the observed aircraft activity on 12 June 2025 within Indian territorial airspace, based on compiled data from naval surveillance systems and field reports. All contact was evaluated and determined to be of Friendly nature.  

        a.	Aircraft types observed were exclusively of Indian origin, with no detection of foreign or unidentified aerial assets.

        b.	Time-stamped reports indicated consistent aircraft movement in strategic sectors across Indian coastal boundaries, confirming a pattern of presence across key naval areas of responsibility (AOR).

            (1) Aerial maneuvers reported included advanced combat training routines:

                (a) Dogfighting simulations consistent with air-to-air engagement exercises.  

                (b) Split-S and Immelmann Turn maneuvers observed in multiple sectors.  

                (c) Barrel Rolls, Scissors maneuvers, and High-G turns performed—indicative of evasive and tactical training operations.

                (d) Zoom climbs observed, matching profiles typical of high-speed vertical ascent training.

            (2) All maneuvers occurred within authorized Indian airspace. Proximity to high-value installations remained within standard operational norms.

    2.  No hostile intent or unauthorized incursion was detected. All aircraft maintained expected communication and transponder compliance with the Indian Naval Aviation Command.  

    3.  The pattern of erratic but deliberate movement, high-speed turns, and advanced aerobatics aligns with known combat-readiness training routines or specialized reconnaissance tasks.  

    4.  Continued observation and logging are recommended to maintain situational awareness and flag any deviations from anticipated friendly patterns.

    5.  No escalation or response action is warranted at this time.

            I. M. COMMANDING

    Copy to:  
    Command Admin (N1/N04C)  
    Programs Integrator (N5)  
    CNRSW Chief of Staff  
    Operations (N3)  
    Information Technology (N6)  
    Tenant Commands  
    Public Works (N4)  
    QOL Director (N9)  
    NAVSUPPACT ANYWHERE

    '''''
    <|im_end|>
    <|im_start|>user
    Report: {report}
    <|im_end|>
    <|im_start|>assistant
    """
    temp=0.5
    max_tokens=1024

    response=llm.create_completion(
        prompt=report_conversion_prompt,
        temperature=temp,
        max_tokens=max_tokens
    )

    result=response['choices'][0]['text']
    result=result.replace("[/INST]", "").replace("'''", "").strip()
    
    return result

In [75]:
formatted=convert_report_to_pdf(state)

Llama.generate: 2368 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   81066.31 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   84506.85 ms /   377 runs   (  224.16 ms per token,     4.46 tokens per second)
llama_perf_context_print:       total time =   85658.90 ms /   378 tokens


In [76]:
print(formatted)

From:	Naval Command
     To:	Here B. Recipient, Organization
     Via:	(1) Here B. Intermediary, Organization

     Subj:	OPERATIONAL REPORT: CHINESE SUBMARINE ACTIVITY ANALYSIS (12 JUN 2025)

     Ref:	(a) Reports by Arnab (ID 2002)

     Encl:	(1) Operational Report: Chinese Submarine Activity Analysis (12 Jun 2025)

     1.  This operational report provides an analysis of Chinese submarine activity on 12 June 2025, as reported by Arnab (ID 2002). The submarines were observed conducting Silent Running and Depth Excursion maneuvers, indicative of advanced underwater warfare training or strategic reconnaissance. All reported positions were within Chinese territorial waters, indicating continued operational focus in these areas.

     2.  The activity of Chinese submarines (ID 2002) suggests an unknown asset operating in our Area of Responsibility (AOR), requiring careful observation and analysis.

     3.  The submarines' presence in critical coastal regions suggests ongoing operationa

In [25]:

def markdown_to_pdf(md_text: str, output_path: str):
    # Convert markdown to HTML
    html = markdown.markdown(md_text)

    # Save HTML to temp file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as tmp:
        tmp.write(html.encode("utf-8"))
        tmp_path = tmp.name

    # Convert HTML to PDF
    pdfkit.from_file(tmp_path, output_path)

    # Cleanup
    os.remove(tmp_path)


In [24]:
markdown_to_pdf("**HELLO**", "report_html.pdf")

In [19]:
markdown_to_pdf(state["report"], "report.pdf")

In [ ]:
def pdf_result(state, file_path: str = "generated_report.pdf") -> bytes:
    formatted_report=convert_report_to_pdf(state)

    temp_html_file_path = None
    temp_pdf_file_path = None

    try:
        html_content = markdown.markdown(formatted_report)

        with tempfile.NamedTemporaryFile(delete=False, suffix=".html", mode="w", encoding="utf-8") as tmp_html:
            tmp_html.write(html_content)
            temp_html_file_path = tmp_html.name
        print(f"Temporary HTML file created: {temp_html_file_path}")

        if file_path:
            # If user wants to save to a specific file, use that path
            output_pdf_actual_path = file_path
            return_bytes = False
        else:
                # If user wants bytes, create another temporary file for PDF output
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf", mode="wb") as tmp_pdf:
                temp_pdf_file_path = tmp_pdf.name
            output_pdf_actual_path = temp_pdf_file_path
            return_bytes = True

        print(f"PDF output path for wkhtmltopdf: {output_pdf_actual_path}")
        
        pdfkit.from_file(temp_html_file_path, output_pdf_actual_path)
        print(f"PDF generated by pdfkit successfully at: {os.path.abspath(output_pdf_actual_path)}")

        if return_bytes:
            # Read the generated PDF file into bytes
            with open(output_pdf_actual_path, "rb") as f:
                pdf_bytes = f.read()
            return pdf_bytes
        else:
            # PDF was saved to disk, return None as per function signature
            return None

    except FileNotFoundError:
        # This specific error usually means wkhtmltopdf is not found
        error_msg = "Error: 'wkhtmltopdf' executable not found. Please ensure it's installed and in your system PATH."
        print(error_msg)
        raise RuntimeError(error_msg) from None # Re-raise for Streamlit to catch
    except Exception as e:
        print(f"An error occurred during PDF generation: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for debugging
        raise # Re-raise the exception to be caught by Streamlit's try/except
    finally:
        # Clean up temporary files
        if temp_html_file_path and os.path.exists(temp_html_file_path):
            os.remove(temp_html_file_path)
            print(f"Cleaned up temporary HTML file: {temp_html_file_path}")
        if temp_pdf_file_path and os.path.exists(temp_pdf_file_path):
            os.remove(temp_pdf_file_path)
            print(f"Cleaned up temporary PDF file: {temp_pdf_file_path}")




In [142]:
def router(state: State)->State:
    '''
    Decides what is to be done with user query, and assigns relevant route
    handles state["route"]
    '''
    print("Routing action to be taken")
    question=state["question"]
    information=state["result"]

    print("Initialised answer")
    routing_prompt=f"""
    <|im_start|>system
    You are a general-purpose AI that helps people with questions.

    Given a question, your job is to categorize it into one of three categories:
    1. report: For when the user instructs, that instruct for report generation.
    2. data: For when the user requests for more data.
    3. analysis: For when the user requests for more analysis.
    4. distance: For when the user requests to find distance.
    5. general: For all other questions.
    Your response should be one word only.

    <|im_end|>
    <|im_start|>user
    Question: {question}
    <|im_end|>
    <|im_start|>assistant
    """
    temp=0.5
    max_tokens=100
    print("Creating response")
    response=llm.create_completion(
        prompt=routing_prompt,
        temperature=temp,
        max_tokens=max_tokens
    )
    print("Created Response")
    result=response['choices'][0]['text']
    result=result.replace("[/INST]", "").replace("'''", "").strip()
    print(result)
    state["route"]=result
    print("finished routing")
    return state


In [147]:
state["question"]="I need more data."

In [148]:
print(state["question"])

I need more data.


In [149]:
print(state["result"])

[(19.63267, 70.3389, '(19.63267, 70.3389)', 'Arnab', 'Erratic movements', 'Friendly', 'air', 'Indian', 'Inside Indian Waters', 'jawaharlal nehru port (nhava sheva)', '12:06:47'), (19.3789, 70.2477, '(19.3789, 70.2477)', 'Arnab', 'Erratic movements', 'Friendly', 'air', 'Indian', 'Inside Indian Waters', 'jawaharlal nehru port (nhava sheva)', '12:05:47'), (19.444, 70.2316, '(19.444, 70.2316)', 'Arnab', 'Erratic movements', 'Friendly', 'air', 'Indian', 'Inside Indian Waters', 'jawaharlal nehru port (nhava sheva)', '12:04:47'), (19.5589, 70.16, '(19.5589, 70.16)', 'Arnab', 'Erratic movements', 'Friendly', 'air', 'Indian', 'Inside Indian Waters', 'jawaharlal nehru port (nhava sheva)', '12:03:47'), (19.5856, 67.2376, '(19.5856, 67.2376)', 'Arnab', 'Erratic movements', 'Friendly', 'air', 'Indian', 'Inside Indian Waters', 'port qasim', '12:02:47')]


In [150]:
llm.reset()
router(state)

Routing action to be taken
Initialised answer
Creating response


llama_perf_context_print:        load time =  116983.60 ms
llama_perf_context_print: prompt eval time =    4740.11 ms /   144 tokens (   32.92 ms per token,    30.38 tokens per second)
llama_perf_context_print:        eval time =      89.06 ms /     1 runs   (   89.06 ms per token,    11.23 tokens per second)
llama_perf_context_print:       total time =    4832.18 ms /   145 tokens


Created Response
data
finished routing


{'db': <langchain_community.utilities.sql_database.SQLDatabase at 0x2a219410850>,
 'db_info': '\nCREATE TABLE "JANES_data" (\n\tlatitude FLOAT, \n\tlongitude FLOAT, \n\tlocation_name TEXT, \n\tlocation_country TEXT, \n\tlocation TEXT\n)\n\n/*\n3 rows from JANES_data table:\nlatitude\tlongitude\tlocation_name\tlocation_country\tlocation\n18.9393\t72.8445\tmumbai\tIndia\t(18.9393, 72.8445)\n19.0056\t72.8163\tjawaharlal nehru port (nhava sheva)\tIndia\t(19.0056, 72.8163)\n17.6856\t83.216\tvisakhapatnam\tIndia\t(17.6856, 83.216)\n*/\n\n\nCREATE TABLE "OTAS_data" (\n\tid BIGINT, \n\tname TEXT, \n\tlatitude FLOAT, \n\tlongitude FLOAT, \n\trange FLOAT, \n\tbearing FLOAT, \n\tcourse FLOAT, \n\tspeed FLOAT, \n\taltitude BIGINT, \n\tdepth BIGINT, \n\treported_by TEXT, \n\tcomment TEXT, \n\thostility TEXT, \n\tcategory TEXT, \n\tnationality TEXT, \n\tlocation_wrt_naval_borders TEXT, \n\tclosest_point_of_mil_interest TEXT, \n\ttime TEXT, \n\tlocation TEXT\n)\n\n/*\n3 rows from OTAS_data table:\nid

In [151]:
from geopy.distance import geodesic
def distance_between_locations(target_location, POI_location):
    '''
    Calculates distance between two given locations, likely between target be observed, and points of military interest
    input: latitude, longitude of target, and latitude, longitude of POI
    output: distance
    '''
    distance=geodesic(target_location, POI_location).kilometers
    return distance

In [155]:
print(distance_between_locations(target_location=([82.3, 84.2]), POI_location=([-70.1, -54.2])))

18329.492909296718


In [169]:
state["question"]="Find distance between new york and delhi. Delhi's latitude and longitude is 28.74 and 77.10."

In [21]:
print(state["db_info"])


CREATE TABLE "JANES_data" (
	latitude FLOAT, 
	longitude FLOAT, 
	location_name TEXT, 
	location_country TEXT, 
	location TEXT
)

/*
3 rows from JANES_data table:
latitude	longitude	location_name	location_country	location
18.9393	72.8445	mumbai	india	(18.9393, 72.8445)
19.0056	72.8163	jawaharlal nehru port (nhava sheva)	india	(19.0056, 72.8163)
17.6856	83.216	visakhapatnam	india	(17.6856, 83.216)
*/


CREATE TABLE "OTAS_data" (
	id BIGINT, 
	name TEXT, 
	latitude FLOAT, 
	longitude FLOAT, 
	range FLOAT, 
	bearing FLOAT, 
	course FLOAT, 
	speed FLOAT, 
	altitude BIGINT, 
	depth BIGINT, 
	reported_by TEXT, 
	comment TEXT, 
	hostility TEXT, 
	category TEXT, 
	nationality TEXT, 
	location_wrt_naval_borders TEXT, 
	closest_point_of_mil_interest TEXT, 
	time TEXT, 
	target_location TEXT
)

/*
3 rows from OTAS_data table:
id	name	latitude	longitude	range	bearing	course	speed	altitude	depth	reported_by	comment	hostility	category	nationality	location_wrt_naval_borders	closest_point_of_mil_inte

In [32]:
def write_sql_query(state: State)->State:
    llm.reset()
    dialect=state["db"].dialect
    top_k=5
    table_info=state["db_info"]
    input=state["question"]

    prompt_template=f"""
    <|im_start|>system

    ###TASK###
    Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.
    
    ###INSTRUCTIONS###:
    Never query for all the columns from a specific table, only ask for a the few relevant columns given the question. Generate only one query.
    Pay attention to use only the column names, and their values that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Prefer structured filters using equality.
    Only use the following tables:
    {table_info}

    ###SCHEMA DESCRIPTION###
    {table_info}

    ###USER FEEDBACK###
    You should avoid fuzzy pattern matching with LIKE '%chinese%' unless strictly necessary.
    You should not generate multiple queries.
    
    <|im_end|>
    <|im_start|>user
    Question: {input}
    <|im_end|>
    <|im_start|>assistant
    """

    temp=0.3
    max_tokens=300

    response=llm.create_completion(
        prompt=prompt_template,
        temperature=temp,
        max_tokens=max_tokens
    )

    result=response['choices'][0]['text']
    result=result.replace("[/INST]", "").replace("'''", "").replace("```", "").strip()
    result = re.sub(r'\b(submarines?|ships?|aircraft|helicopters?)\b',
           lambda m: {'submarine': 'subsurface', 'submarines': 'subsurface',
                      'ship': 'surface', 'ships': 'surface',
                      'aircraft': 'air', 'helicopter': 'air', 'helicopters': 'air'}[m.group(0).lower()],
           result, flags=re.IGNORECASE)
    result=result.lower()
    state["query"]=result
    state["question"]=input
    llm.reset()
    print("---------------------------------\n\n")
    print(state["query"])
    print("---------------------------------\n\n")
    return state

In [33]:
state: State={}

In [37]:
state["question"]="Generate a report on what utkarsh has reported."
state=assign_db(state)
state=write_sql_query(state)
state=execute_query(state)
print("---------------------------question-------------------------\n\n")
print(state["question"])
print("---------------------------generated query-------------------------\n\n")
print(state["query"])
print("---------------------------generated response-------------------------\n\n")
print(state["result"])

llama_perf_context_print:        load time =   58072.58 ms
llama_perf_context_print: prompt eval time =   56713.74 ms /  1351 tokens (   41.98 ms per token,    23.82 tokens per second)
llama_perf_context_print:        eval time =    2054.66 ms /    15 runs   (  136.98 ms per token,     7.30 tokens per second)
llama_perf_context_print:       total time =   58791.85 ms /  1366 tokens


---------------------------------


select * from otas_data where reported_by = 'utkarsh';
---------------------------------


---------------------------question-------------------------


Generate a report on what utkarsh has reported.
---------------------------generated query-------------------------


select * from otas_data where reported_by = 'utkarsh';
---------------------------generated response-------------------------


[(2004, 'ghazi', 20.44, 69.23, 204.6, 300.7, 127.5, 5.4, 0, 0, 'utkarsh', 'seems to be approaching territorial waters', 'hostile', 'surface', 'pakistani', 'inside indian waters', 'jawaharlal nehru port (nhava sheva)', '11:25:58', '(20.44, 69.23)')]


In [ ]:
from langsmith import Client
client=Client(api_key=)

In [2]:
print("**OPERATIONAL REPORT: UTKARSH ACTIVITY ANALYSIS (19 JUN 2025)**\n\n     **TO:** Naval Command\n     **TIME AND DATE OF GENERATION OF REPORT:** 19th June 2025 15:24:27\n\n     **SUBJECT:** Assessment of Utkarsh Class Corvette (ID 3002) Activity\n\n     **SUMMARY:**\n     Utkarsh (ID 3002) has been observed conducting a series of maneuvers within Indian territorial waters. The vessel exhibited a pattern of rapid accelerations and decelerations, coupled with significant course changes, indicative of advanced tactical training or complex operational drills.\n\n     **OBSERVATIONS & ANALYSIS:**\n\n     1.  **Identity & Status:** Target ID 3002, identified as a Utkarsh class Corvette of Indian nationality. Categorized as Active. The consistently friendly status indicates an operational asset.\n\n     2.  **Geographic & Temporal Span:** Data points range from 19:00:00 to 19:10:00 IST, all within Indian Waters, proximate to **Gujarat Coast**. This signifies continued presence in a critical coastal region.\n\n     3.  **Movement Patterns:**\n         * **Course Volatility:** The Corvette's course shows significant variance (from 315.3 to 45.4 degrees True North). This, coupled with the observation comments, strongly suggests non-linear flight paths consistent with advanced tactical training, complex patrol patterns, or operational drills.\n         * **Speed Fluctuations:** Speed varied considerably, from a low of 45 knots to a high of 120 knots. Such accelerations and decelerations are typical of advanced naval maneuvers.\n         * **Range & Altitude:** Range varied, while altitude remained constant at 30 feet (nominal for surface-level operations, implying it's not conducting ground-level observation).\n\n     4.  **Points of Interest:** The closest point of military interest remains consistently 'gujarat coast', reinforcing its operational focus in that vicinity.\n\n     5.  **Hostility Assessment:** Confirmed Active status throughout all reports, mitigating immediate threat assessment. However, the erratic movements and variable speeds are indicative of a trained asset, capable of handling a variety of naval scenarios.\n\n     **CONCLUSION:**\n     The Utkarsh (ID 3002) is engaging in what appears to be standard operational maneuvers, likely training or specialized operational drills, within authorized Indian waters. The erratic movements and variable speeds are characteristic of advanced naval training. Continued monitoring is advised to confirm operational intent and detect any deviation from expected friendly patterns.\n\n     **NOTE:** Further analysis or reporting may be warranted based on the specific maneuvers observed or if the vessel deviates from its friendly status.")

**OPERATIONAL REPORT: UTKARSH ACTIVITY ANALYSIS (19 JUN 2025)**

     **TO:** Naval Command
     **TIME AND DATE OF GENERATION OF REPORT:** 19th June 2025 15:24:27

     **SUBJECT:** Assessment of Utkarsh Class Corvette (ID 3002) Activity

     **SUMMARY:**
     Utkarsh (ID 3002) has been observed conducting a series of maneuvers within Indian territorial waters. The vessel exhibited a pattern of rapid accelerations and decelerations, coupled with significant course changes, indicative of advanced tactical training or complex operational drills.

     **OBSERVATIONS & ANALYSIS:**

     1.  **Identity & Status:** Target ID 3002, identified as a Utkarsh class Corvette of Indian nationality. Categorized as Active. The consistently friendly status indicates an operational asset.

     2.  **Geographic & Temporal Span:** Data points range from 19:00:00 to 19:10:00 IST, all within Indian Waters, proximate to **Gujarat Coast**. This signifies continued presence in a critical coastal region.

